# Web-scraping Canadian Postal Codes
We use BeautifulSoup to obtain the table of Canadian postal codes from [Wikipedia](http://www.wikizero.biz/index.php?q=aHR0cHM6Ly9lbi53aWtpcGVkaWEub3JnL3dpa2kvTGlzdF9vZl9wb3N0YWxfY29kZXNfb2ZfQ2FuYWRhOl9N)

In [1]:
# Import the necessary libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re

In [2]:
# Assign variable 'table' to the URL containing the needed table
table = requests.get(r"http://www.wikizero.biz/index.php?q=aHR0cHM6Ly9lbi53aWtpcGVkaWEub3JnL3dpa2kvTGlzdF9vZl9wb3N0YWxfY29kZXNfb2ZfQ2FuYWRhOl9N").text

# Create a BeautifulSoup instance
soup = BeautifulSoup(table, "lxml")
# print(soup.prettify())

### Extracting the table
After inspecting the webpage, I realized that the table of interest has the **tbody** tag  
The other associated tags are:  
* *th* ---- Header tag
* *td* ---- Row tag

In [3]:
# Extract the table from the 'soup' instance
Table = soup.find('tbody')
# print(Table.prettify())

In [4]:
# From Table, we can extract the headers which have the 'th' tag
header = [Columns.text for Columns in Table.find_all('th')]
header[-1] = header[-1][:-1] # remove new line (\n) character at the end
header[0] = 'PostalCode'
print(header)

# From Table, we can extract the rows which have the 'td' tag
body = [body.text for body in Table.find_all('td')]
body = np.array(body).reshape([-1,3])

['PostalCode', 'Borough', 'Neighbourhood']


### Create the initial DataFrame

In [5]:
df = pd.DataFrame.from_records(body)
df.columns = header

# Delete the last character (\n) of every row
df['Neighbourhood'] = df.Neighbourhood.str.replace("\n", "")
df.head()

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Clean the data
The necessary steps as specified in the instructions are carried out

In [6]:
# delete rows that do not have a Borough
df = df[df.Borough != 'Not assigned']

# Group neighbourhoods that have the same borough
df_final = pd.DataFrame(df.groupby(['PostalCode', 'Borough'])['Neighbourhood'].sum())

df_final.reset_index(level=df_final.index.names, inplace=True)
df_final.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,RougeMalvern
1,M1C,Scarborough,Highland CreekRouge HillPort Union
2,M1E,Scarborough,GuildwoodMorningsideWest Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


#### Use regular expressions to perform string operations
I noticed that the text in the **Neighbourhood** coulumn are not properly formatted after the *join* operation  
There is not comma and space between neighborhoods.  
Using regular expressions in a for loop, this problem was solved

In [7]:

splits = [re.sub(r"(?<=\w)([A-Z])", r", \1", df_final.Neighbourhood.values[i]) for i, x in enumerate(df_final.Neighbourhood.values)]
df_final['Neighbourhood'] = splits
df_final.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [8]:
print(df_final.shape)

(103, 3)
